In [1]:
import pandas as pd
import networkx as nx
import pygraphviz as pgv
from shortest_path import SignedPathSolver
from pagerank import PageRankSolver


## Shortest path

### Sign consistency

## Showcase with Trynska data

In [2]:
collectri = pd.read_csv('network_collectri.sif', sep='\t')
data_col = collectri.pop('data')
collectri.insert(2, 'data', data_col)
collectri.rename(columns={'source': '# source'}, inplace=True)
collectri.to_csv('collectri_network_sscp.sif', sep='\t', index=None)
collectri

,# source,target,data
0,CALM1,TRPC1,-1
1,CALM3,TRPC1,-1
2,CALM2,TRPC1,-1
3,CAV1,TRPC1,1
4,MDFI,TRPC1,-1
...,...,...,...
81749,NFKB,hsa-miR-143-3p,1
81750,AP1,hsa-miR-206,1
81751,NFKB,hsa-miR-21-5p,1
81752,NFKB,hsa-miR-224-5p,1


In [3]:
collectri[collectri['# source'] == 'TGFB1']

,# source,target,data
3650,TGFB1,NOS2,-1
4391,TGFB1,APP,1
4772,TGFB1,TGFBR2,1
4773,TGFB1,TGFBR3,1
4774,TGFB1,TGFBR1,1
12589,TGFB1,TGFBR1_TGFBR2,1
12614,TGFB1,EGFR,1
12615,TGFB1,SMAD3,1
14179,TGFB1,SMAD7,1
14743,TGFB1,SPARC,1


In [4]:
target_df = pd.read_csv('downstream_hits.tsv', sep='\t', header=None)
target_df.drop(1, inplace=True, axis=1)
target_df

,0,2
0,CINP,+
1,ATP1B1,+
2,DNAJC12,+
3,SCCPDH,+
4,IDS,+
...,...,...
95,IKZF1,-
96,SDHD,+
97,RASGRP3,+
98,BIRC6,-


In [5]:
target_df[2].replace({'+': 1, '-': -1}, inplace=True)

target_dict = target_df.set_index(0)[2].to_dict()

In [6]:
G = nx.read_weighted_edgelist('collectri_network_sscp.sif', delimiter = '\t', create_using = nx.DiGraph)

# 2. Iterate through each edge and 3. Set attributes
for u, v, data in G.edges(data=True):
    weight = data['weight']
    data['sign'] = 1 if weight >= 0 else -1
    data['weight'] = abs(weight)

G.get_edge_data('TGFB1', 'KLF2') #none

In [7]:
G_solver = SignedPathSolver(G)
G_solver.source_dict = {'TGFB1': 1, 'TGFB2': 1, 'TGFB3': 1, 'IL2': 1}
G_solver.target_dict = target_dict
G_solver.shortest_paths()

In [8]:
G_solver.sign_consistency_check()

g2_solver = SignedPathSolver(G)
g2_solver.shortest_paths_res = G_solver.shortest_paths_res
g2_solver.shortest_sc_paths_res = G_solver.shortest_sc_paths_res
g2_solver.source_dict = G_solver.source_dict
g2_solver.target_dict = G_solver.target_dict

In [9]:
G_solver.visualize_graph(G_solver.shortest_paths_res, "shortest_paths", is_sign_consistent=False, export=True)

In [10]:
G_solver.visualize_graph(G_solver.shortest_sc_paths_res, "shortest_paths", is_sign_consistent=True, export=True)

## PageRank

In [7]:
P_solver = PageRankSolver(G)
P_solver.source_dict = {'TGFB1': 1, 'TGFB2': 1, 'TGFB3': 1, 'IL2': 1}
P_solver.target_dict = target_dict

In [8]:
P_solver.pagerank_solver(personalize_for='source')

In [9]:
P_solver.pagerank_solver(personalize_for='target')

In [10]:
P_solver.threshold = 0.005
P_solver.compute_filtered_shortest_paths(sign_consistent=False)
P_solver.visualize_pagerank("PageRank_0_005", is_sign_consistent=False)

2


In [11]:
P_solver.compute_filtered_shortest_paths(sign_consistent=True)
P_solver.visualize_pagerank("PageRank_0_005", is_sign_consistent=True)

2


In [12]:
P_solver.shortest_sc_paths_res

[['TGFB1', 'SMAD3', 'MYC', 'KLF7'],
 ['TGFB1', 'SMAD2', 'SP1', 'SKIL'],
 ['TGFB1', 'SMAD2', 'MYC', 'TP53', 'KLF2'],
 ['TGFB1', 'SMAD7', 'MYC', 'TP53', 'KLF2'],
 ['TGFB1', 'SMAD2', 'SP1', 'TP53', 'KLF2'],
 ['TGFB1', 'SMAD3', 'ESR1', 'TP53', 'KLF2'],
 ['TGFB1', 'SMAD3', 'AR', 'TP53', 'KLF2'],
 ['TGFB1', 'CDKN1A', 'E2F1', 'RELA', 'KLF2'],
 ['TGFB1', 'SMAD7', 'E2F1', 'RELA', 'KLF2'],
 ['TGFB1', 'SMAD3', 'AR', 'RELA', 'KLF2'],
 ['TGFB1', 'SMAD3', 'EGR2'],
 ['TGFB2', 'TGFBR1', 'SMAD4', 'MYC', 'KLF7'],
 ['TGFB2', 'TGFBR2', 'SMAD4', 'MYC', 'KLF7'],
 ['TGFB2', 'TGFBR1', 'SMAD3', 'MYC', 'KLF7'],
 ['TGFB2', 'TGFBR2', 'SMAD3', 'MYC', 'KLF7'],
 ['TGFB2', 'TGFBR1', 'SMAD2', 'MYC', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR2', 'SMAD2', 'MYC', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR2', 'SMAD7', 'MYC', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR1', 'SMAD2', 'SP1', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR2', 'SMAD2', 'SP1', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR1', 'SMAD3', 'ESR1', 'TP53', 'KLF2'],
 ['TGFB2', 'TGFBR2', 'SMAD3', 'ESR1'